In [1]:
import pandas as pd

# EOBMF Dataset

In [20]:
north_east = "https://www.irs.gov/pub/irs-soi/eo1.csv"
mid_atlantic = "https://www.irs.gov/pub/irs-soi/eo2.csv"
gulf_pacific_coasts = "https://www.irs.gov/pub/irs-soi/eo3.csv"

In [31]:
df = pd.read_csv(north_east)

In [ ]:
df.head(10)

In [26]:
df.columns

Index(['EIN', 'NAME', 'ICO', 'STREET', 'CITY', 'STATE', 'ZIP', 'GROUP',
       'SUBSECTION', 'AFFILIATION', 'CLASSIFICATION', 'RULING',
       'DEDUCTIBILITY', 'FOUNDATION', 'ACTIVITY', 'ORGANIZATION', 'STATUS',
       'TAX_PERIOD', 'ASSET_CD', 'INCOME_CD', 'FILING_REQ_CD',
       'PF_FILING_REQ_CD', 'ACCT_PD', 'ASSET_AMT', 'INCOME_AMT', 'REVENUE_AMT',
       'NTEE_CD', 'SORT_NAME'],
      dtype='object')

In [23]:
df['STATE'].unique()

array(['MA', 'NY', 'NJ', 'ME', 'NH', 'VT', 'CT', 'RI'], dtype=object)

# What is the 'grain'
EOBMF Extract:
- My understanding is this dataset is cumulative list of every unique tax-exempt business entity in the US. This list is updated on a monthly basis from the IRS. 

# Broad Categories of Dataset

EOBMF Extract has:
- Geographical information
- Organizational information
- Most recent, high level tax information